# Frank-Wolfe Algorithm

*Mar, 2017 by K.Wu*

In this report, network are presented as a CSV file:

- [**/data/Frank-Wolfe-algo-data.csv**](https://github.com/wklchris/Reports/blob/master/data/Frank-Wolfe-algo-data.csv): All links of the network.

First, here are the packages needed in this report. Install: 

- Python 3
- numpy package
- pandas package

to continue.

In [1]:
import os
import numpy as np
import pandas as pd

## Introduction

In this report, we have a network like below:

<img src="https://raw.githubusercontent.com/wklchris/Reports/master/data/Frank-Wolfe-algo-network.png" alt="Network Picture" style="width: 500px;"/>

The free flow travel time $t_a$ and capacities $c_a$ for each type of link are:

- Black links: $t_a = 10, c_a = 50$
- Red links: $t_a = 30, c_a = 200$
- Yellow links: $t_a = 15, c_a = 200$
- Cyan links: $t_a = 0, c_a = 2000$

We also have the O-D table of this network, shown as below:

In [2]:
ODarr = pd.DataFrame([[100, 200, 300, 50],
                      [100, 200, 200, 100],
                      [300, 50, 100, 100],
                      [200, 500, 50, 200]])

# Rename columns & rows
ODarr.columns = list(range(201, 205))
ODarr.index = list(range(101, 105))

ODarr

,201,202,203,204
101,100,200,300,50
102,100,200,200,100
103,300,50,100,100
104,200,500,50,200


## Network Representation



In [3]:
linkarr = pd.read_csv(r"{}/data/Frank-Wolfe-algo-data.csv".format(os.getcwd()))
linkarr.head()

,start,end,type
0,1,2,black
1,2,3,black
2,3,4,black
3,14,15,black
4,15,16,black


We can modify the link array according to the type:

In [4]:
linkarr["fftime"] = 0
linkarr["cap"] = 0

# Set the FreeFlowTraveltime and Capacity of each link
linkarr.loc[linkarr.type == "black", "fftime"] = 10
linkarr.loc[linkarr.type == "black", "cap"] = 50
linkarr.loc[linkarr.type == "yellow", "fftime"] = 15
linkarr.loc[linkarr.type == "yellow", "cap"] = 200
linkarr.loc[linkarr.type == "red", "fftime"] = 30
linkarr.loc[linkarr.type == "red", "cap"] = 200
linkarr.loc[linkarr.type == "cyan", "fftime"] = 0
linkarr.loc[linkarr.type == "cyan", "cap"] = 2000

linkarr.head(3)

,start,end,type,fftime,cap
0,1,2,black,10,50
1,2,3,black,10,50
2,3,4,black,10,50


## Application Scenarios

As for the costs, we use BPR function to compute the cost on each link. Cost on link $a$, called $t_a$, is a function of $x_a$:

$$ t_a(x_a) = t_a^0 \left[1 + \alpha\left(\frac{x_a}{c_a'}\right)^\beta\right] $$

where $\alpha = 0.15, \beta = 4, c_a' = 0.9c_a$. And we can define it as a function:

In [5]:
def bpr(xa, fft, ca, alpha=0.15, beta=4, ca_factor=0.9):
    """
    Compute the cost when the flow is 'xa' with BPR function.
    """
    return fft * (1 + alpha * (np.divide(xa, ca_factor * ca) ** beta))

As for the shortest path finding, we use the function defined in the [Label-correcting-algo.ipynb](https://github.com/wklchris/Reports/blob/master/Label-correcting-algo.ipynb):

In [6]:
def label_correcting_algo(dt, ori_node, des_node, do_return=False):
    """
    Find the shortest path from Origin to Destination under a constant-link-costs network.
    
    Args:
        dt: Network representation. At least 3 columns:
              "start": start nodes of links
              "end": end nodes of links
              "cost": constant costs of links
        ori_node: Origin node.
        des_node: Destination node.
        do_return: Boolean. 
            If True, Return a dataframe as described below.
            If False, Return the shortest path string instead.
        
    Returns:
        A dataframe of two columns:
            "Front-Node": The node visited before current node on the shortest path.
            "Distance": Total distance from origin to current node.
    """
    # Convert all labels to string
    ori = str(ori_node)
    des = str(des_node)
    dt[["start", "end"]] = dt[["start", "end"]].astype(str) 
    
    # Initialization
    nodes = set(dt.loc[:,"start"].unique()) | set(dt.loc[:,"end"].unique())
    dist = {}.fromkeys(nodes, np.inf)
    dist[ori] = 0
    points = {}.fromkeys(nodes, ori)
    iter_set = {ori}
    
    # Main Algo
    while iter_set:
        i = iter_set.pop()  # Randomly pop out a node i
        A_i = dt[dt.start == i]
        for row in A_i.index: 
            j = A_i.loc[:, "end"][row]
            c_ij = A_i.loc[:, "cost"][row]
            if dist[j] > dist[i] + c_ij:
                dist[j] = dist[i] + c_ij
                points[j] = i
                iter_set = iter_set | set([j])  # Union
    
    # Print & Return the Answer
    x = pd.concat([pd.Series(points), pd.Series(dist)], axis=1)
    x.columns = ["Front-node", "Costs"]

    current_node = des
    front_node = ""
    sp = [des]
    while front_node != ori:
        front_node = str(x.loc[current_node, "Front-node"])
        # sp = "{} -> {}".format(front_node, sp)
        sp.insert(0, front_node)
        current_node = front_node
    
    # sp.append(x.loc[des, "Costs"])
    # sp = "From node {} to node {}, total cost: {}\n{}\n".format(ori, des, x.loc[des, "Costs"], sp)
    if do_return:
        print(sp)
        return x
    else:
        return sp

### Q1

*Find the user equilibrium (UE) flow pattern, and compute the total travel time experienced by all users (this is not to be confused with the sum of the integral of travel times). Plot the convergence curves (total network link flow vs iteration number, total network travel time vs iteration number) and discuss your results.*

-----


First, we use free flow travel time to find the initial shortest path. So the cost under this situation equals free flow travel time:

In [7]:
bprlinkarr = linkarr.copy()
bprlinkarr["cost"] = bprlinkarr.fftime

bprlinkarr.head()

,start,end,type,fftime,cap,cost
0,1,2,black,10,50,10
1,2,3,black,10,50,10
2,3,4,black,10,50,10
3,14,15,black,10,50,10
4,15,16,black,10,50,10


In [8]:
def solvex4eq(q0, q1, q2, q3, q4, theta=0.01, a=0, b=1):
    """
    Bi-section method to solve quartic equation.
    """
    y = lambda x: q0 + q1 * x + q2 * x ** 2 + q3 * x ** 3 + q4 * x ** 4
    fa = y(a)
    fb = y(b)
    
    if fa * fb > 0:
        return list(map(y, np.linspace(a, b, 20)))
    elif (fa == 0):
        return a
    elif (fb == 0):
        return b
    
    while (b - a > theta):
        c = (b + a) / 2
        fc = y(c)
        if (fc == 0):
            return c
        elif (fc * fa < 0):
            b = c
        elif (fc * fb < 0):
            a = c
    
    return c

solvex4eq(-0.2, 0.4, 2.4, 2.8, 1)

0.1953125

And we need another function to do the AON assignment to the network, i.e. assign the demand between OD to the shortest path:

In [9]:
def frank_wolfe(x, OD_arr, thre=1, alpha=0.15):
    ori_lst = OD_arr.index
    des_lst = OD_arr.columns
    err = thre + 1
    iters = 0
    
    while (err > thre):
        iters += 1        
        new_col = "xa{}".format(iters)
        x[new_col] = 0
        
        # AON assignment
        for i in range(len(ori_lst)):
            ori = ori_lst[i]
            for j in range(len(des_lst)):
                des = des_lst[j]
                demand = OD_arr.at[ori, des]
                splst = label_correcting_algo(x, ori, des)
                while len(splst) > 1:
                    current_node = splst.pop()
                    front_node = splst[-1]
                    x.loc[(x.start == front_node) & (x.end == current_node), new_col] += demand
        
        # Update the costs
        x.cost = bpr(x[new_col], x.fftime, x.cap)
        
        # Line search
        if (iters > 1):
            direction = x[new_col] - x[old_col]
            
            # Compute the coefficients of the equation of k. 0 <= k <= 1.
            p0 = (0.15 * alpha) / ((0.9 * x.cap) ** 4) * x[old_col] ** 4 + x.fftime
            p1 = 4 * (0.15 * alpha) / ((0.9 * x.cap) ** 4) * np.multiply(x[old_col] ** 3, direction)
            p2 = 6 * (0.15 * alpha) / ((0.9 * x.cap) ** 4) * np.multiply(x[old_col] ** 2, direction ** 2)
            p3 = 4 * (0.15 * alpha) / ((0.9 * x.cap) ** 4) * np.multiply(x[old_col], direction ** 3)
            p4 = (0.15 * alpha) / ((0.9 * x.cap) ** 4) * x[old_col] ** 4
            k = solvex4eq(sum(p0), sum(p1), sum(p2), sum(p3), sum(p4))
            if (type(k) != type(1)):
                print("Solving error. Iteration: {}".format(iters))
                return k
            x[new_col] = x[old_col] + k * (x.loc[:, new_col] - x[old_col])
            
            # Compute the error
            err = np.sqrt(sum((x[new_col] - x[old_col]) ** 2)) / sum(x[old_col])
            
        # Move the iteration forward    
        old_col = "xa{}".format(iters)
        
        if (iters > 10):
            print("Too many iterations.")
            return -1000
            
    return x

frank_wolfe(bprlinkarr, ODarr)

Solving error. Iteration: 2


[4477.7123536897716,
 3700.5722896982638,
 3042.9536122983259,
 2492.0832184479095,
 2035.9242264084273,
 1663.1759757447558,
 1363.2740273252341,
 1126.3901633216622,
 943.43238720930481,
 806.04492376689018,
 706.60821907660556,
 638.23894052410344,
 594.78997679850136,
 570.85043789237,
 561.74565510175421,
 563.53718102615699,
 573.02278956853934,
 587.7364759353336,
 605.94845663642718,
 626.66516948517483]